In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sqlite3 as sql
import geopandas as gpd
import folium
from shapely.geometry import point
from folium.plugins import MarkerCluster

%matplotlib inline

## Imported CSV files to dataframes

In [ ]:
incident_response = pd.read_csv('../Data/charts_clean.csv')
incident_response.head()

In [ ]:
calls_address = pd.read_csv('../Data/calls_clean.csv')
calls_address.head()

In [ ]:
geo_clean = pd.read_csv('../Data/geocodeclean.csv')
geo_clean.head()

## Merged geo data to calls/address

In [ ]:
geo_calls_merged = pd.merge(geo_clean, calls_address, left_on = 'address', right_on = 'Address')
#geo_calls_merged.drop(['address'], axis=1, inplace = True)
geo_calls_merged.info()

## Consolidated priority calls and dropped excess priority columns

In [ ]:
incident_response['Priority_2'] = incident_response['Priority_2'] + incident_response['Priority_2L'] + incident_response['Priority_2M'] + incident_response['Priority_2N'] + incident_response['Priority_2T'] + incident_response['Priority_2R']
incident_response['Priority_3'] = incident_response['Priority_3'] + incident_response['Priority_3H'] + incident_response['Priority_3M'] + incident_response['Priority_3Q']
incident_response.drop(['Priority_2L', 'Priority_2M', 'Priority_2N', 'Priority_2T', 'Priority_2R', 'Priority_3H', 'Priority_3M', 'Priority_3Q'], axis=1, inplace = True)
incident_response.head()

In [ ]:
incident_response.info()

In [ ]:
calls_per_precinct = incident_response.groupby('Precinct')['Total Incidents'].sum().to_frame()
calls_per_precinct = calls_per_precinct.sort_values('Total Incidents', ascending = False)
calls_per_precinct

In [ ]:
calls_per_precinct['Total Incidents'].plot(kind = 'bar',
                                    figsize = (17,8), color =['cornflowerblue', 'coral', 'pink'])

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
incident_type_precinct = pd.crosstab(geo_calls_merged.Precinct, geo_calls_merged['Incident Type'])
incident_type_precinct.plot(kind='bar', stacked=True, ax=ax, color = ['#1b9e77', '#a9f971', '#fdaa48','#6890F0','#A890F0'])
plt.legend(bbox_to_anchor = (1, 1.015), loc = 'upper left')
plt.title('Incident Type Per Precinct',
         fontsize = 14,
         fontweight = 'bold')
plt.show()

In [ ]:
precinct_priority = incident_response.groupby('Precinct')['Priority_1', 'Priority_2', 'Priority_3'].sum()
precinct_priority 

In [ ]:
precinct_priority.info()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
precinct_priority.plot(kind='bar', stacked=True, ax=ax, color = ['#1b9e77', '#a9f971', '#fdaa48'])
plt.legend(bbox_to_anchor = (1, 1.015), loc = 'upper left')
plt.title('Priority Calls Per Precinct',
         fontsize = 14,
         fontweight = 'bold')
plt.show()

In [ ]:
conn = sql.connect('../Data/census.sqlite')
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [ ]:
available_tables = (cur.fetchall())
print(available_tables)

In [ ]:
cur.execute('SELECT * FROM B01001 LIMIT 5;')

pop_age_sex = (cur.fetchall())

In [ ]:
type(pop_age_sex)

In [ ]:
pop_age_sex = pd.read_sql("SELECT * FROM B01001;", conn)
pop_age_sex

In [ ]:
#pop_age_sex.to_excel('sample_test.xlsx')

In [ ]:
#geo_calls_merged.to_excel('crime_data.xlsx')

In [ ]:
cur.execute('SELECT * FROM B03002 LIMIT 5;')

table_2 = (cur.fetchall())

In [ ]:
table_2

In [ ]:
police_precinct_map = gpd.read_file('../Data/Police District.geojson')
police_precinct_map

In [ ]:
census_tract = gpd.read_file('../Data/census_tract/tl_2021_47_tract.shp')
census_tract.info()

In [ ]:
mask_census = census_tract['COUNTYFP'] == '037'
census_filtered = census_tract[mask_census]
census_filtered

In [ ]:
leg_kwds = {'title': 'precinct', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 1}
police_precinct_map.plot(column = 'precinct', figsize = (8,8), edgecolor = 'black', cmap = 'Set3', legend = True, legend_kwds = leg_kwds)

In [ ]:
test = police_precinct_map.plot(column = 'precinct', figsize = (8,8), edgecolor = 'black', cmap = 'Set1', legend = True)
census_filtered.plot(ax = test, edgecolor = 'black', alpha = 0.7)

In [ ]:
census_filtered['center'] = census_filtered.geometry.centroid

In [ ]:
census_filtered.dtypes

In [ ]:
tract_center = census_filtered.drop(['geometry'], axis=1)
tract_center = tract_center.rename(columns = {'center':'geometry'})
tract_center

In [ ]:
tract_center.plot()

In [ ]:
joint_tract = gpd.sjoin(tract_center, police_precinct_map, op= 'within')

In [ ]:
ax = police_precinct_map.plot(edgecolor = "black", figsize = (8,8), cmap = "Set3")
joint_tract.plot(ax = ax, edgecolor = "white")

In [ ]:
joint_tract.info()